In [1]:
from __future__ import absolute_import, division, print_function

import os, sys, re, gc, pickle, operator, shutil, copy
import time, datetime
from collections import Counter, defaultdict

from math import floor, ceil
import numpy as np
import pandas as pd
from scipy.stats import spearmanr
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.utils import shuffle

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, Dataset, DataLoader, Sampler

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm, tqdm_notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings(action='once')

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from pytorch_pretrained_bert import convert_tf_checkpoint_to_pytorch
from pytorch_pretrained_bert import BertTokenizer, BertForSequenceClassification, BertAdam, BertConfig
from pytorch_pretrained_bert.modeling import BertModel, BertPreTrainedModel
device = torch.device('cuda')

/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [3]:
DATA_DIR = "/kaggle/input/google-quest-challenge/"
# BERT_MODEL_PATH = '/kaggle/input/bert-pretrained-models/cased_L-12_H-768_A-12/cased_L-12_H-768_A-12/'
BERT_MODEL_PATH = '/kaggle/input/bert-pretrained-models/uncased_L-12_H-768_A-12/uncased_L-12_H-768_A-12/'
# BERT_MODEL_PATH = '/kaggle/input/bert-pretrained-models/uncased_L-24_H-1024_A-16/uncased_L-24_H-1024_A-16/'

MODEL_PATH1 = '../input/gq-bert-cls-1/'
MODEL_PATH2 = '../input/gq-bert-pool-aug/'
# MODEL_PATH2 = '../input/gq-2bert-pool-1/'
# MODEL_PATH2_ap = '../input/gq-2bert-pool-2/'
# MODEL_PATH2 = '../input/gq-bert-pool-exp/'
# MODEL_PATH2 = '../input/gq-bert-pool/'
# MODEL_PATH2 = '../input/gq-bert-pool-cased/'
# MODEL_PATH2 = '../input/gq-bert-large-pool/'
MODEL_PATH3 = '../input/gq-bert-2pool-exp/'
# MODEL_PATH3_1 = '../input/gq-bert-2pool-exp3/'
# MODEL_PATH3_2 = '../input/gq-bert-2pool-exp4/'
# MODEL_PATH3_3 = '../input/gq-bert-2pool-exp5/'
MODEL_PATH3_1 = '../input/gq-bert-2pool/'
MODEL_PATH3_2 = '../input/gq-bert-2pool-exp8/'
MODEL_PATH3_3 = '../input/gq-bert-2pool-alltrunc/'
# MODEL_PATH3 = '../input/gq-bert-2pool-cased/'
MODEL_PATH4 = '../input/gq-bert-cls-pool/'
MODEL_PATH5 = '../input/gq-bert-4pool-exp/'
# MODEL_PATH5 = '../input/gq-bert-last4/'
MODEL_PATH6 = '../input/gq-bert-2catpool/'
MODEL_PATH7 = '../input/gq-bert-3catpool/'
MODEL_PATH8 = '../input/gq-bert-3pool/'


SEED = 2019

MAX_SEQUENCE_LENGTH = 512
batch_size = 16

In [4]:
# Seed for randomness in pytorch
def seed_torch(seed=SEED):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

### Functions of data preprocessing and pytorch datasets

In [5]:
# # Thanks to https://www.kaggle.com/akensert/bert-base-tf2-0-minimalistic
# def trim_and_tokenize(title, question, answer, max_sequence_length, tokenizer,
#                       t_max_len=18, q_max_len=245, a_max_len=244):

#     tq_sep = tokenizer.tokenize("Details:")
#     t = tokenizer.tokenize(title)
#     q = tokenizer.tokenize(question)
#     a = tokenizer.tokenize(answer)
    
#     t_len = len(t)
#     q_len = len(q)
#     a_len = len(a)

#     if (t_len+q_len+a_len+5) > max_sequence_length:
        
#         if t_max_len > t_len:
#             t_new_len = t_len
#             a_max_len = a_max_len + floor((t_max_len - t_len)/2)
#             q_max_len = q_max_len + ceil((t_max_len - t_len)/2)
#         else:
#             t_new_len = t_max_len
      
#         if a_max_len > a_len:
#             a_new_len = a_len 
#             q_new_len = q_max_len + (a_max_len - a_len)
#         elif q_max_len > q_len:
#             a_new_len = a_max_len + (q_max_len - q_len)
#             q_new_len = q_len
#         else:
#             a_new_len = a_max_len
#             q_new_len = q_max_len
            
#         if t_new_len+a_new_len+q_new_len+5 != max_sequence_length:
#             raise ValueError("New sequence length should be %d, but is %d" 
#                              % (max_sequence_length, (t_new_len+a_new_len+q_new_len+5)))
        
# #         t = t[-t_new_len:]
# #         q = q[-q_new_len:]
# #         a = a[-a_new_len:]
#         t = t[:t_new_len]
#         q = q[:q_new_len]
#         a = a[:a_new_len]
    
#     return t, tq_sep, q, a


# def trim_and_tokenize_2(title, question, answer, tokenizer,
#                       t_max_len=20, q_max_len=300, a_max_len=300):

#     t = tokenizer.tokenize(title)
#     q = tokenizer.tokenize(question)
#     a = tokenizer.tokenize(answer)
    
#     t = t[:t_max_len]
#     q = q[:q_max_len]
#     a = a[:a_max_len]
    
#     return t, q, a


# # Tokenizing the lines to BERT token
# def convert_lines(df, columns, max_sequence_length, tokenizer):
#     all_tokens = []
#     segment_ids = []   # representing segmentation of sentence A and B
    
#     for _, instance in df[columns].iterrows():
#         t, q, a = instance.question_title, instance.question_body, instance.answer
#         t, tq_sep, q, a = trim_and_tokenize(t, q, a, max_sequence_length, tokenizer)
#         tokens = ["[CLS]"] + t + tq_sep + q + ["[SEP]"] + a + ["[SEP]"]
#         all_tokens.append(tokenizer.convert_tokens_to_ids(tokens))
#         segment_ids.append([0]*(len(t)+len(tq_sep)+len(q)+2) + [1]*(len(a)+1))
    
#     return np.array(all_tokens), np.array(segment_ids)


# def convert_lines_2(df, columns, tokenizer):
#     q_all_tokens = []
#     a_all_tokens = []
#     q_segment_ids = []   # representing segmentation of sentence A and B
    
#     for _, instance in df[columns].iterrows():
#         t, q, a = instance.question_title, instance.question_body, instance.answer
#         t, q, a = trim_and_tokenize_2(t, q, a, tokenizer)
#         q_tokens = ["[CLS]"] + t + ["[SEP]"] + q + ["[SEP]"]
#         a_tokens = ["[CLS]"] + a + ["[SEP]"]
#         q_all_tokens.append(tokenizer.convert_tokens_to_ids(q_tokens))
#         a_all_tokens.append(tokenizer.convert_tokens_to_ids(a_tokens))
#         q_segment_ids.append([0]*(len(t)+2) + [1]*(len(q)+1))
    
#     return np.array(q_all_tokens), np.array(a_all_tokens), np.array(q_segment_ids)

In [6]:
# Thanks to https://www.kaggle.com/akensert/bert-base-tf2-0-minimalistic
def trim_and_tokenize(title, question, answer, max_sequence_length, tokenizer,
                      trunc_mode='head', t_max_len=18, q_max_len=245, a_max_len=244):
    
    assert trunc_mode in {"head", "tail", "mix"}
    need_trunc = False

    tq_sep = tokenizer.tokenize("Details:")
    t = tokenizer.tokenize(title)
    q = tokenizer.tokenize(question)
    a = tokenizer.tokenize(answer)
    
    t_len = len(t)
    q_len = len(q)
    a_len = len(a)

    if (t_len+q_len+a_len+5) > max_sequence_length:
        need_trunc = True
        
        if t_max_len > t_len:
            t_new_len = t_len
            a_max_len = a_max_len + floor((t_max_len - t_len)/2)
            q_max_len = q_max_len + ceil((t_max_len - t_len)/2)
        else:
            t_new_len = t_max_len
      
        if a_max_len > a_len:
            a_new_len = a_len 
            q_new_len = q_max_len + (a_max_len - a_len)
        elif q_max_len > q_len:
            a_new_len = a_max_len + (q_max_len - q_len)
            q_new_len = q_len
        else:
            a_new_len = a_max_len
            q_new_len = q_max_len
            
        if t_new_len+a_new_len+q_new_len+5 != max_sequence_length:
            raise ValueError("New sequence length should be %d, but is %d" 
                             % (max_sequence_length, (t_new_len+a_new_len+q_new_len+5)))
        
        if trunc_mode == "head":
            t = t[:t_new_len]
            q = q[:q_new_len]
            a = a[:a_new_len]
        if trunc_mode == "tail":
            t = t[-t_new_len:]
            q = q[-q_new_len:]
            a = a[-a_new_len:]
        if trunc_mode == "mix":
            def trunc_seq(seq, seq_max_len, trunc_ratio=0.6):
                maj_len = int(seq_max_len * trunc_ratio)
                return seq[:maj_len] + seq[-(seq_max_len-maj_len):]
            t = trunc_seq(t, t_new_len)
            q = trunc_seq(q, q_new_len)
            a = trunc_seq(a, a_new_len)
    
    return t, tq_sep, q, a, need_trunc


# Tokenizing the lines to BERT token
def convert_lines(df, columns, max_sequence_length, tokenizer, trunc_mode='head', misc_trunc=False, target=None):
    all_tokens = []
    segment_ids = []   # representing segmentation of sentence A and B
    if target is not None:
        labels = []
    
    for ind, (_, instance) in enumerate(df[columns].iterrows()):
        
        title, question, answer = instance.question_title, instance.question_body, instance.answer
        t, tq_sep, q, a, need_trunc = trim_and_tokenize(title, question, answer,
                                                        max_sequence_length, tokenizer, trunc_mode=trunc_mode)
        tokens = ["[CLS]"] + t + tq_sep + q + ["[SEP]"] + a + ["[SEP]"]
        all_tokens.append(tokenizer.convert_tokens_to_ids(tokens))
        segment_ids.append([0]*(len(t)+len(tq_sep)+len(q)+2) + [1]*(len(a)+1))
        if target is not None:
            labels.append(target[ind])
        
        if need_trunc and misc_trunc:
            t, tq_sep, q, a, _ = trim_and_tokenize(title, question, answer,
                                                   max_sequence_length, tokenizer, trunc_mode='tail')
            tokens = ["[CLS]"] + t + tq_sep + q + ["[SEP]"] + a + ["[SEP]"]
            all_tokens.append(tokenizer.convert_tokens_to_ids(tokens))
            segment_ids.append([0]*(len(t)+len(tq_sep)+len(q)+2) + [1]*(len(a)+1))

            t, tq_sep, q, a, _ = trim_and_tokenize(title, question, answer,
                                                   max_sequence_length, tokenizer, trunc_mode='mix')
            tokens = ["[CLS]"] + t + tq_sep + q + ["[SEP]"] + a + ["[SEP]"]
            all_tokens.append(tokenizer.convert_tokens_to_ids(tokens))
            segment_ids.append([0]*(len(t)+len(tq_sep)+len(q)+2) + [1]*(len(a)+1))

            if target is not None:
                labels.extend([target[ind], target[ind]])
    
    if target is not None:
        return np.array(all_tokens), np.array(segment_ids), np.array(labels)
    return np.array(all_tokens), np.array(segment_ids)

In [7]:
# Prepare dataset and dataloader

class QuestQAs(Dataset):

    def __init__(self, tokenized_comments, segment_ids, targets=None, split=None, maxlen=256):
        self.comments = tokenized_comments
        self.segment_ids = segment_ids
        self.targets = targets
        self.split = split
        assert self.split in {'train', 'valid', 'test'}
        self.maxlen = maxlen

    def __getitem__(self, index):
        comment = self.comments[index]
        segment_id = self.segment_ids[index]
        if self.targets is not None:
            target = self.targets[index]
            return comment, segment_id, torch.FloatTensor(target)
        else:
            return comment, segment_id

    def __len__(self):
        return len(self.comments)

    def get_lens(self):
        lengths = np.fromiter(
            ((min(self.maxlen, len(seq))) for seq in self.comments),
            dtype=np.int32)
        return lengths

    def collate_fn(self, batch):
        """
        Collate function for sequence bucketing
        Note: this need not be defined in this Class, can be standalone.

        :param batch: an iterable of N sets from __getitem__()
        :return: a tensor of comments, and targets
        """

        if self.split in ('train', 'valid'):
            comments, segment_ids, targets = zip(*batch)
        else:
            comments, segment_ids = zip(*batch)

        lengths = [len(c) for c in comments]
        maxlen = max(lengths)
        padded_comments, padded_seg_ids = [], []
        for i, (c, s) in enumerate(zip(comments, segment_ids)):
            padded_comments.append(c+[0]*(maxlen - lengths[i]))
            padded_seg_ids.append(s +[0]*(maxlen - lengths[i]))

        if self.split in ('train', 'valid'):
            return torch.LongTensor(padded_comments), torch.LongTensor(padded_seg_ids), torch.stack(targets)
        else:
            return torch.LongTensor(padded_comments), torch.LongTensor(padded_seg_ids)


class QuestQAs2(Dataset):

    def __init__(self, tokenized_q_comments, tokenized_a_comments, q_segment_ids, targets=None, split=None, maxlen=256):
        self.q_comments = tokenized_q_comments
        self.a_comments = tokenized_a_comments
        self.q_segment_ids = q_segment_ids
        self.targets = targets
        self.split = split
        assert self.split in {'train', 'valid', 'test'}
        self.maxlen = maxlen

    def __getitem__(self, index):
        q_comment = self.q_comments[index]
        a_comment = self.a_comments[index]
        q_segment_id = self.q_segment_ids[index]
        if self.targets is not None:
            target = self.targets[index]
            return q_comment, a_comment, q_segment_id, torch.FloatTensor(target)
        else:
            return q_comment, a_comment, q_segment_id

    def __len__(self):
        return len(self.q_comments)

    def get_lens(self):
        lengths = np.fromiter(
            ((min(self.maxlen, len(seq))) for seq in self.q_comments),
            dtype=np.int32)
        return lengths

    def collate_fn(self, batch):
        """
        Collate function for sequence bucketing
        Note: this need not be defined in this Class, can be standalone.

        :param batch: an iterable of N sets from __getitem__()
        :return: a tensor of comments, and targets
        """

        if self.split in ('train', 'valid'):
            q_comments, a_comments, q_segment_ids, targets = zip(*batch)
        else:
            q_comments, a_comments, q_segment_ids = zip(*batch)

        q_lengths = [len(c) for c in q_comments]
        a_lengths = [len(c) for c in a_comments]
        q_maxlen = max(q_lengths)
        a_maxlen = max(a_lengths)
        padded_q_comments, padded_a_comments, padded_q_seg_ids = [], [], []
        for i, (q, a, s) in enumerate(zip(q_comments, a_comments, q_segment_ids)):
            padded_q_comments.append(q+[0]*(q_maxlen - q_lengths[i]))
            padded_a_comments.append(a+[0]*(a_maxlen - a_lengths[i]))
            padded_q_seg_ids.append(s +[0]*(q_maxlen - q_lengths[i]))

        if self.split in ('train', 'valid'):
            return torch.LongTensor(padded_q_comments), torch.LongTensor(padded_a_comments), \
                    torch.LongTensor(padded_q_seg_ids), torch.stack(targets)
        else:
            return torch.LongTensor(padded_q_comments), torch.LongTensor(padded_a_comments), torch.LongTensor(padded_q_seg_ids)


class BucketSampler(Sampler):

    def __init__(self, data_source, sort_lens, bucket_size=None, batch_size=1024, shuffle_data=True):
        super().__init__(data_source)
        self.sort_lens = sort_lens
        self.bucket_size = bucket_size if bucket_size is not None else len(sort_lens)
        self.batch_size = batch_size
        self.shuffle = shuffle_data
        
        self.weights = None
        if not shuffle_data:
            self.index = self.prepare_buckets()
        else:
            self.index = None

    def set_weights(self, weights):
        assert weights >= 0
        self.weights = weights / np.sum(weights)

    def get_reverse_indexes(self):
        indexes = np.zeros((len(self.index),), dtype=np.int32)
        for i, j in enumerate(self.index):
            indexes[j] = i
        return indexes

    def __len__(self):
        return len(self.sort_lens)

    def __iter__(self):
        indices = None
        if self.weights is not None:
            total = len(self.sort_lens)
            indices = np.random.choice(total, (total,), p=self.weights)
        if self.shuffle:
            self.index = self.prepare_buckets(indices)
        return iter(self.index)

    def prepare_buckets(self, indices=None):
        lengths = - self.sort_lens  # negative sign for descending sort later
        assert self.bucket_size % self.batch_size == 0 or self.bucket_size == len(lengths)

        if indices is None:
            if self.shuffle:
                indices = shuffle(np.arange(len(lengths), dtype=np.int32))
                lengths = lengths[indices]
            else:
                indices = np.arange(len(lengths), dtype=np.int32)

        def divide_chunks(l, n):
            """bucket iterator
            l: data to be divided
            n: number of chunks
            """
            if n == len(l):
                yield np.arange(len(l), dtype=np.int32), l
            else:
                # looping till length of l
                for i in range(0, len(l), n):
                    data = l[i:i + n]
                    yield np.arange(i, i + len(data), dtype=np.int32), data

        new_indices = []
        extra_batch_idx = None
        for chunk_indices, chunk in divide_chunks(lengths, self.bucket_size):
            # sort indices in bucket by descending order of lengths
            indices_sorted = chunk_indices[np.argsort(chunk)]

            batch_idxes = []
            for _, batch_idx in divide_chunks(indices_sorted, self.batch_size):
                if len(batch_idx) == self.batch_size:
                    batch_idxes.append(batch_idx.tolist())
                else:
                    assert extra_batch_idx is None
                    assert batch_idx is not None
                    extra_batch_idx = batch_idx.tolist()

            # shuffling batches within buckets
            if self.shuffle:
                batch_idxes = shuffle(batch_idxes)
            for batch_idx in batch_idxes:
                new_indices.extend(batch_idx)

        if extra_batch_idx is not None:
            new_indices.extend(extra_batch_idx)

        if not self.shuffle:
            self.original_indices = np.argsort(indices_sorted).tolist()
        return indices[new_indices]


def prepare_loader(x, seg_ids, y=None, batch_size=None, split=None):
    assert split in {'train', 'valid', 'test'}
    dataset = QuestQAs(x, seg_ids, y, split, MAX_SEQUENCE_LENGTH)
    if split == 'train':
        sampler = BucketSampler(dataset, dataset.get_lens(),
                                bucket_size=batch_size*20, batch_size=batch_size)
        return DataLoader(dataset, batch_size=batch_size, sampler=sampler,
                          collate_fn=dataset.collate_fn)
    else:
        sampler = BucketSampler(dataset, dataset.get_lens(),
                                batch_size=batch_size, shuffle_data=False)
        return DataLoader(dataset, batch_size=batch_size, sampler=sampler,
                          collate_fn=dataset.collate_fn), sampler.original_indices


def prepare_loader_2(q, a, q_seg_ids, y=None, batch_size=None, split=None):
    assert split in {'train', 'valid', 'test'}
    dataset = QuestQAs2(q, a, q_seg_ids, y, split, MAX_SEQUENCE_LENGTH)
    if split == 'train':
        sampler = BucketSampler(dataset, dataset.get_lens(),
                                bucket_size=batch_size*20, batch_size=batch_size)
        return DataLoader(dataset, batch_size=batch_size, sampler=sampler,
                          collate_fn=dataset.collate_fn)
    else:
        sampler = BucketSampler(dataset, dataset.get_lens(),
                                batch_size=batch_size, shuffle_data=False)
        return DataLoader(dataset, batch_size=batch_size, sampler=sampler,
                          collate_fn=dataset.collate_fn), sampler.original_indices

### Model

In [8]:
class BertQAClassification(BertPreTrainedModel):
    def __init__(self, config, num_labels):
        super(BertQAClassification, self).__init__(config)
        self.num_labels = num_labels
        self.bert_q = BertModel(config)
        self.bert_a = BertModel(config)
        self.dropout = nn.Dropout(0.2)
        self.classifier = nn.Linear(config.hidden_size*2, num_labels)
        self.apply(self.init_bert_weights)

    def forward(self, input_q_ids, input_a_ids, q_token_type_ids=None, a_token_type_ids=None, 
                q_attention_mask=None, a_attention_mask=None):
        seq_q_output, _ = self.bert_q(input_q_ids, q_token_type_ids, q_attention_mask, output_all_encoded_layers=False)
        seq_a_output, _ = self.bert_a(input_a_ids, a_token_type_ids, a_attention_mask, output_all_encoded_layers=False)
        sequence_output = torch.cat((seq_q_output, seq_a_output), 1)
        avg_pool = torch.mean(sequence_output, 1)
        max_pool, _ = torch.max(sequence_output, 1)
        pooled_output = torch.cat((avg_pool, max_pool), 1)
        return self.classifier(self.dropout(pooled_output))

In [9]:
class BertForCLSClassification(BertPreTrainedModel):
    def __init__(self, config, num_labels):
        super(BertForCLSClassification, self).__init__(config)
        self.num_labels = num_labels
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(0.2)
        self.classifier = nn.Linear(config.hidden_size, num_labels)
        self.apply(self.init_bert_weights)

    def forward(self, input_ids, token_type_ids=None, attention_mask=None):
        _, pooled_output = self.bert(input_ids, token_type_ids, attention_mask, output_all_encoded_layers=False)
        return self.classifier(self.dropout(pooled_output))


class BertForPoolClassification(BertPreTrainedModel):
    def __init__(self, config, num_labels):
        super(BertForPoolClassification, self).__init__(config)
        self.num_labels = num_labels
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(0.2)
#         self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(config.hidden_size*2, num_labels)
        self.apply(self.init_bert_weights)

    def forward(self, input_ids, token_type_ids=None, attention_mask=None):
        sequence_output, _ = self.bert(input_ids, token_type_ids, attention_mask, output_all_encoded_layers=False)
        avg_pool = torch.mean(sequence_output, 1)
        max_pool, _ = torch.max(sequence_output, 1)
        pooled_output = torch.cat((avg_pool, max_pool), 1)
        return self.classifier(self.dropout(pooled_output))


class BertLastTwoClassification(BertPreTrainedModel):
    def __init__(self, config, num_labels):
        super(BertLastTwoClassification, self).__init__(config)
        self.num_labels = num_labels
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(config.hidden_size*4, num_labels)
        self.apply(self.init_bert_weights)

    def forward(self, input_ids, token_type_ids=None, attention_mask=None):
        encoded_layers, _ = self.bert(input_ids, token_type_ids, attention_mask, output_all_encoded_layers=True)
        seq_op1 = encoded_layers[-1]
        seq_op2 = encoded_layers[-2]
        avg_pool1 = torch.mean(seq_op1, 1)
        max_pool1, _ = torch.max(seq_op1, 1)
        avg_pool2 = torch.mean(seq_op2, 1)
        max_pool2, _ = torch.max(seq_op2, 1)
        pooled_output = torch.cat((avg_pool1, max_pool1, avg_pool2, max_pool2), 1)
        return self.classifier(self.dropout(pooled_output))


class BertCLSLastTwoClassification(BertPreTrainedModel):
    def __init__(self, config, num_labels):
        super(BertCLSLastTwoClassification, self).__init__(config)
        self.num_labels = num_labels
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(config.hidden_size*5, num_labels)
        self.apply(self.init_bert_weights)

    def forward(self, input_ids, token_type_ids=None, attention_mask=None):
        encoded_layers, pooled_cls = self.bert(input_ids, token_type_ids, attention_mask, output_all_encoded_layers=True)
        seq_op1 = encoded_layers[-1]
        seq_op2 = encoded_layers[-2]
        avg_pool1 = torch.mean(seq_op1, 1)
        max_pool1, _ = torch.max(seq_op1, 1)
        avg_pool2 = torch.mean(seq_op2, 1)
        max_pool2, _ = torch.max(seq_op2, 1)
        pooled_output = torch.cat((avg_pool1, max_pool1, avg_pool2, max_pool2, pooled_cls), 1)
        return self.classifier(self.dropout(pooled_output))


class BertLastFourClassification(BertPreTrainedModel):
    def __init__(self, config, num_labels):
        super(BertLastFourClassification, self).__init__(config)
        self.num_labels = num_labels
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(config.hidden_size*8, num_labels)
        self.apply(self.init_bert_weights)

    def forward(self, input_ids, token_type_ids=None, attention_mask=None):
        encoded_layers, pooled_cls = self.bert(input_ids, token_type_ids, attention_mask, output_all_encoded_layers=True)
        seq_op1 = encoded_layers[-1]
        seq_op2 = encoded_layers[-2]
        seq_op3 = encoded_layers[-3]
        seq_op4 = encoded_layers[-4]
        avg_pool1 = torch.mean(seq_op1, 1)
        max_pool1, _ = torch.max(seq_op1, 1)
        avg_pool2 = torch.mean(seq_op2, 1)
        max_pool2, _ = torch.max(seq_op2, 1)
        avg_pool3 = torch.mean(seq_op3, 1)
        max_pool3, _ = torch.max(seq_op3, 1)
        avg_pool4 = torch.mean(seq_op4, 1)
        max_pool4, _ = torch.max(seq_op4, 1)
        pooled_output = torch.cat((avg_pool1, max_pool1, avg_pool2, max_pool2,
                                   avg_pool3, max_pool3, avg_pool4, max_pool4), 1)
        return self.classifier(self.dropout(pooled_output))
    
    
class BertCatLastTwoClassification(BertPreTrainedModel):
    def __init__(self, config, num_labels):
        super(BertCatLastTwoClassification, self).__init__(config)
        self.num_labels = num_labels
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(config.hidden_size*4, num_labels)
        self.apply(self.init_bert_weights)

    def forward(self, input_ids, token_type_ids=None, attention_mask=None):
        encoded_layers, _ = self.bert(input_ids, token_type_ids, attention_mask, output_all_encoded_layers=True)
        seq_op1 = encoded_layers[-1]   # (N, T, D)
        seq_op2 = encoded_layers[-2]   # (N, T, D)
        seq_cat = torch.cat((seq_op1, seq_op2), -1)   # (N, T, 2D)
        avg_pool = torch.mean(seq_cat, 1)   # (N, 2D)
        max_pool, _ = torch.max(seq_cat, 1)   # (N, 2D)
        pooled_output = torch.cat((avg_pool, max_pool), 1)   # (N, 4D)
        return self.classifier(self.dropout(pooled_output))
    

class BertLastThreeClassification(BertPreTrainedModel):
    def __init__(self, config, num_labels):
        super(BertLastThreeClassification, self).__init__(config)
        self.num_labels = num_labels
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(0.35)
        self.classifier = nn.Linear(config.hidden_size*6, num_labels)
        self.apply(self.init_bert_weights)

    def forward(self, input_ids, token_type_ids=None, attention_mask=None):
        encoded_layers, _ = self.bert(input_ids, token_type_ids, attention_mask, output_all_encoded_layers=True)
        seq_op1 = encoded_layers[-1]
        seq_op2 = encoded_layers[-2]
        seq_op3 = encoded_layers[-3]
        avg_pool1 = torch.mean(seq_op1, 1)
        max_pool1, _ = torch.max(seq_op1, 1)
        avg_pool2 = torch.mean(seq_op2, 1)
        max_pool2, _ = torch.max(seq_op2, 1)
        avg_pool3 = torch.mean(seq_op3, 1)
        max_pool3, _ = torch.max(seq_op3, 1)
        pooled_output = torch.cat((avg_pool1, max_pool1, avg_pool2, max_pool2, avg_pool3, max_pool3), 1)
        return self.classifier(self.dropout(pooled_output))


class BertCatLastThreeClassification(BertPreTrainedModel):
    def __init__(self, config, num_labels):
        super(BertCatLastThreeClassification, self).__init__(config)
        self.num_labels = num_labels
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(config.hidden_size*6, num_labels)
        self.apply(self.init_bert_weights)

    def forward(self, input_ids, token_type_ids=None, attention_mask=None):
        encoded_layers, _ = self.bert(input_ids, token_type_ids, attention_mask, output_all_encoded_layers=True)
        seq_op1 = encoded_layers[-1]   # (N, T, D)
        seq_op2 = encoded_layers[-2]   # (N, T, D)
        seq_op3 = encoded_layers[-3]   # (N, T, D)
        seq_cat = torch.cat((seq_op1, seq_op2, seq_op3), -1)   # (N, T, 3D)
        avg_pool = torch.mean(seq_cat, 1)   # (N, 3D)
        max_pool, _ = torch.max(seq_cat, 1)   # (N, 3D)
        pooled_output = torch.cat((avg_pool, max_pool), 1)   # (N, 6D)
        return self.classifier(self.dropout(pooled_output))

In [10]:
class BertGroup1(nn.Module):
    def __init__(self, bert):
        super(BertGroup1, self).__init__()
        self.embeddings = bert.embeddings
        self.bert_encoder = copy.deepcopy(bert).encoder.layer[:8]
        
    def forward(self, input_ids, token_type_ids=None, attention_mask=None):
        if attention_mask is None:
            attention_mask = torch.ones_like(input_ids)
        if token_type_ids is None:
            token_type_ids = torch.zeros_like(input_ids)
        
        extended_attention_mask = attention_mask.unsqueeze(1).unsqueeze(2)
        extended_attention_mask = extended_attention_mask.to(dtype=next(self.parameters()).dtype) # fp16 compatibility
        extended_attention_mask = (1.0 - extended_attention_mask) * -10000.0
        
        hidden_states = self.embeddings(input_ids, token_type_ids)
        for layer_module in self.bert_encoder:
            hidden_states = layer_module(hidden_states, extended_attention_mask)
            
        return hidden_states


class BertGroup2(nn.Module):
    def __init__(self, bert):
        super(BertGroup2, self).__init__()
        self.bert_encoder = copy.deepcopy(bert).encoder.layer[:8]
        
    def forward(self, hidden_states, input_ids, attention_mask=None):
        if attention_mask is None:
            attention_mask = torch.ones_like(input_ids)
        
        extended_attention_mask = attention_mask.unsqueeze(1).unsqueeze(2)
        extended_attention_mask = extended_attention_mask.to(dtype=next(self.parameters()).dtype) # fp16 compatibility
        extended_attention_mask = (1.0 - extended_attention_mask) * -10000.0
        
        all_encoder_layers = []
        for layer_module in self.bert_encoder:
            hidden_states = layer_module(hidden_states, extended_attention_mask)
            all_encoder_layers.append(hidden_states)
            
        return all_encoder_layers
    

class BertGroupsClassification(nn.Module):
    def __init__(self, bert, config, num_labels):
        super(BertGroupsClassification, self).__init__()
        self.num_labels = num_labels
        self.bert_group_1 = BertGroup1(bert)
        self.bert_group_2 = BertGroup2(bert)
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(config.hidden_size*4, num_labels)
        
    def forward(self, input_ids, token_type_ids=None, attention_mask=None):
        hidden_states = self.bert_group_1(input_ids, token_type_ids, attention_mask)
        encoded_layers = self.bert_group_2(hidden_states, input_ids, attention_mask)
        seq_op1 = encoded_layers[-1]
        seq_op2 = encoded_layers[-2]
        avg_pool1 = torch.mean(seq_op1, 1)
        max_pool1, _ = torch.max(seq_op1, 1)
        avg_pool2 = torch.mean(seq_op2, 1)
        max_pool2, _ = torch.max(seq_op2, 1)
        pooled_output = torch.cat((avg_pool1, max_pool1, avg_pool2, max_pool2), 1)
        return self.classifier(self.dropout(pooled_output))

### Functions for validation and inference

In [11]:
def compute_rho(labels, preds):
    rhos = []
    for col_label, col_pred in zip(labels.T, preds.T):
        rhos.append(
            spearmanr(col_label, col_pred + np.random.normal(0, 1e-7, col_pred.shape[0])).correlation)
    return np.mean(rhos)


def validate(val_loader, model, val_df, val_original_indices):
    model.eval()
    targets, scores = [], []

    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device=device, dtype=torch.long), y.to(device=device, dtype=torch.float)
            score = model(x, attention_mask=(x>0))
            targets.append(y.cpu().numpy())
            scores.append(torch.sigmoid(score).cpu().numpy())

    targets = np.concatenate(targets)
    scores = np.concatenate(scores)
    val_rho = compute_rho(targets, scores)
    print('{"metric": "Val. Rho", "value": %.4f}' % (val_arho, ))
    
#     val_scores = scores[val_original_indices]
#     return val_scores


def eval_model(model, test_loader):
    model.eval()
    test_scores = []
    with torch.no_grad():
        for x, seg_ids in test_loader:
            x = x.to(device=device, dtype=torch.long)
            seg_ids = seg_ids.to(device=device, dtype=torch.long)
            score = torch.sigmoid(model(x, token_type_ids=seg_ids, attention_mask=(x>0)))
            test_scores.append(score.cpu().numpy())
    return np.concatenate(test_scores)


def eval_model_2(model, test_loader):
    model.eval()
    test_scores = []
    with torch.no_grad():
        for q, a, seg_ids in test_loader:
            q, a = q.to(device=device, dtype=torch.long), a.to(device=device, dtype=torch.long)
            seg_ids = seg_ids.to(device=device, dtype=torch.long)
            score = torch.sigmoid(model(q, a, q_token_type_ids=seg_ids, 
                                  q_attention_mask=(q>0), a_attention_mask=(a>0)))
            test_scores.append(score.cpu().numpy())
    return np.concatenate(test_scores)


class NNAverage(object):
    def __init__(self, model, mu=0.5):
        self.mu = mu
        self.weight_copy = {}
        for name, param in model.named_parameters():
            if param.requires_grad:
                self.weight_copy[name] = 0

    def update(self, model):
        for name, param in model.named_parameters():
            if param.requires_grad:
                self.weight_copy[name] += self.mu * param.data

    def set_weights(self, avg_model):
        for name, param in avg_model.named_parameters():
            if param.requires_grad:
                param.data = self.weight_copy[name]

### Load/preprocess the data, and test the model

In [12]:
def train_val_split(train_df):
#     kf = GroupKFold(n_splits=5)
    kf = KFold(n_splits=5, shuffle=True, random_state=SEED)
#     cv_indices = [(tr_idx, val_idx) for tr_idx, val_idx in kf.split(train_df.question_body, groups=train_df.question_body)]
    cv_indices = [(tr_idx, val_idx) for tr_idx, val_idx in kf.split(train_df)]
    return cv_indices


def load_data():
    train_df = pd.read_csv(DATA_DIR+'train.csv')
    test_df = pd.read_csv(DATA_DIR+'test.csv')
    sub_df = pd.read_csv(DATA_DIR+'sample_submission.csv')
    train_df = pd.read_csv(DATA_DIR+'train.csv')
    output_cols = list(train_df.columns[11:])
    input_cols = list(train_df.columns[[1,2,5]])
    
    test_seq, test_seg_ids = convert_lines(test_df, input_cols, MAX_SEQUENCE_LENGTH, tokenizer)

    return train_df, test_df, sub_df, input_cols, output_cols


def load_and_preproc():
    test_df = pd.read_csv(DATA_DIR+'test.csv')
    sub_df = pd.read_csv(DATA_DIR+'sample_submission.csv')
    train_df = pd.read_csv(DATA_DIR+'train.csv')
    input_cols = list(train_df.columns[[1,2,5]])
    
    test_seq, test_seg_ids = convert_lines(test_df, input_cols, MAX_SEQUENCE_LENGTH, tokenizer)

    return test_seq, test_seg_ids, test_df, sub_df


def load_and_preproc_2():
    test_df = pd.read_csv(DATA_DIR+'test.csv')
    sub_df = pd.read_csv(DATA_DIR+'sample_submission.csv')
    train_df = pd.read_csv(DATA_DIR+'train.csv')
    input_cols = list(train_df.columns[[1,2,5]])
    
    test_q_seq, test_a_seq, test_q_seg_ids = convert_lines_2(test_df, input_cols, tokenizer)

    return test_q_seq, test_a_seq, test_q_seg_ids, test_df, sub_df

In [13]:
%%time

tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_PATH, do_lower_case=True)
# tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_PATH)

# t0 = time.time()
# print('Loading and tokenizing...')
# x_test, seg_test, test_df, sub_df = load_and_preproc()
# x_q_test, x_a_test, seg_q_test, test_df, sub_df = load_and_preproc_2()
train_df, test_df, sub_df, input_cols, output_cols = load_data()
# print('tokenizing complete in {:.0f} seconds.'.format(time.time()-t0))

CPU times: user 5.82 s, sys: 41.4 ms, total: 5.86 s
Wall time: 5.91 s


In [14]:
%%time
t0 = time.time()
print('Tokenizing...')

x_test, seg_test = convert_lines(test_df, input_cols, MAX_SEQUENCE_LENGTH, tokenizer)

x_test_2, seg_test_2 = convert_lines(test_df, input_cols,
                                     MAX_SEQUENCE_LENGTH, tokenizer,
                                     trunc_mode='tail')

print('tokenizing complete in {:.0f} seconds.'.format(time.time()-t0))

Tokenizing...
tokenizing complete in 11 seconds.
CPU times: user 11.3 s, sys: 72 µs, total: 11.3 s
Wall time: 11.3 s


In [15]:
# inference preparation

test_loader, test_original_indices = prepare_loader(x_test, seg_test, batch_size=batch_size, split='test')
test_loader_2, test_original_indices_2 = prepare_loader(x_test_2, seg_test_2, batch_size=batch_size, split='test')
# test_loader, test_original_indices = prepare_loader_2(x_q_test, x_a_test, seg_q_test, batch_size=batch_size, split='test')

In [16]:
torch.cuda.empty_cache()
print(str(torch.cuda.memory_allocated(device)/1e6 ) + 'M')
print(str(torch.cuda.memory_cached(device)/1e6 ) + 'M')

0.0M
0.0M


In [17]:
# BERT configuration file for model loading
bert_config = BertConfig(BERT_MODEL_PATH + 'bert_config.json')

# model setup
torch.cuda.empty_cache()

# print("Building model...")
# # model = BertForCLSClassification(bert_config, num_labels=30)
# model = BertForPoolClassification(bert_config, num_labels=30)
# model = model.to(device)

# models = torch.load(MODEL_PATH+'quest_bert_models.pt')['model']
# model.load_state_dict(models[list(models.keys())[-1]])

In [18]:
# # test inference
# t0 = time.time()
# torch.cuda.empty_cache()

# # val_preds = validate(val_loader, model, val_df, val_original_indices)

# test_preds = eval_model(model, test_loader)[test_original_indices]
# # test_preds = []
# # ckpt_weights = [1.1**e for e in range(len(models))]
# # for state in models.values():
# #     model.load_state_dict(state)
# #     test_preds.append(eval_model(model, test_loader)[test_original_indices])
# # test_preds = np.average(test_preds, weights=ckpt_weights, axis=0)

# time_elapsed = time.time() - t0
# print('time = {:.0f}m {:.0f}s.'.format(time_elapsed // 60, time_elapsed % 60))

In [19]:
# # weight avg of models from diff folds
# print("Building model...")
# model = BertForCLSClassification(bert_config, num_labels=30)
# avgd = NNAverage(model, 1./5)

# for i in range(5):
#     models = torch.load(MODEL_PATH1+f'fld_{i}_quest_bert_models.pt')['model']
#     model.load_state_dict(models[list(models.keys())[-1]])
#     avgd.update(model)

# print('Averaging and inference...')
# avgd.set_weights(model)
# model = model.to(device)
# test_preds = eval_model(model, test_loader)[test_original_indices]

In [20]:
# # model setup
# torch.cuda.empty_cache()

# print("Building model...")
# model = BertForCLSClassification(bert_config, num_labels=30)
# model = model.to(device)

# # test inference
# t0 = time.time()
# torch.cuda.empty_cache()

# test_preds_1 = []
# for i in range(5):
#     models = torch.load(MODEL_PATH1+f'fld_{i}_quest_bert_models.pt')['model']
#     model.load_state_dict(models[list(models.keys())[-1]])
#     test_preds_1.append(eval_model(model, test_loader)[test_original_indices])
# test_preds_1 = np.mean(test_preds_1, 0)
# # test_preds = np.mean(test_preds_1, 0)

# time_elapsed = time.time() - t0
# print('time = {:.0f}m {:.0f}s.'.format(time_elapsed // 60, time_elapsed % 60))

In [21]:
# # model setup
# torch.cuda.empty_cache()

# print("Building model...")
# model = BertQAClassification(bert_config, num_labels=30)
# model = model.to(device)

# # test inference
# t0 = time.time()
# torch.cuda.empty_cache()

# test_preds_2 = []
# for i in range(5):
#     if i < 3:
#         models = torch.load(MODEL_PATH2+f'fld_{i}_quest_bert_models.pt')['model']
#     else:
#         models = torch.load(MODEL_PATH2_ap+f'fld_{i}_quest_bert_models.pt')['model']
#     model.load_state_dict(models[list(models.keys())[-1]])
#     test_preds_2.append(eval_model_2(model, test_loader)[test_original_indices])
# # test_preds_2 = np.mean(test_preds_2, 0)
# test_preds = np.mean(test_preds_2, 0)

# # models = torch.load(MODEL_PATH2+f'quest_bert_models.pt')['model']
# # model.load_state_dict(models[list(models.keys())[-1]])
# # test_preds = eval_model(model, test_loader)[test_original_indices]

# time_elapsed = time.time() - t0
# print('time = {:.0f}m {:.0f}s.'.format(time_elapsed // 60, time_elapsed % 60))

In [22]:
# # model setup
# torch.cuda.empty_cache()

# print("Building model...")
# model = BertForPoolClassification(bert_config, num_labels=30)
# model = model.to(device)

# # test inference
# t0 = time.time()
# torch.cuda.empty_cache()

# test_preds_2 = []
# for i in range(5):
#     models = torch.load(MODEL_PATH2+f'fld_{i}_quest_bert_models.pt')['model']
#     model.load_state_dict(models[list(models.keys())[-1]])
#     test_preds_2.append(eval_model(model, test_loader)[test_original_indices])
# # test_preds_2 = np.mean(test_preds_2, 0)
# test_preds = np.mean(test_preds_2, 0)

# # models = torch.load(MODEL_PATH2+f'quest_bert_models.pt')['model']
# # model.load_state_dict(models[list(models.keys())[-1]])
# # test_preds = eval_model(model, test_loader)[test_original_indices]

# time_elapsed = time.time() - t0
# print('time = {:.0f}m {:.0f}s.'.format(time_elapsed // 60, time_elapsed % 60))

In [23]:
# model setup
torch.cuda.empty_cache()

print("Building model...")
model = BertLastTwoClassification(bert_config, num_labels=30)
model = model.to(device)

# test inference
t0 = time.time()
torch.cuda.empty_cache()

test_preds_3 = []
for i in range(5):
    models = torch.load(MODEL_PATH3+f'fld_{i}_quest_bert_models.pt')['model']
    model.load_state_dict(models[list(models.keys())[-1]])
    test_preds_3.append(eval_model(model, test_loader)[test_original_indices])
test_preds_3 = np.mean(test_preds_3, 0)
# test_preds = np.mean(test_preds_3, 0)

time_elapsed = time.time() - t0
print('time = {:.0f}m {:.0f}s.'.format(time_elapsed // 60, time_elapsed % 60))

Building model...


<All keys matched successfully>

<All keys matched successfully>

<All keys matched successfully>

<All keys matched successfully>

<All keys matched successfully>

time = 0m 33s.


In [24]:
# model setup
torch.cuda.empty_cache()

print("Building model...")
model = BertLastTwoClassification(bert_config, num_labels=30)
model = model.to(device)

# test inference
t0 = time.time()
torch.cuda.empty_cache()

test_preds_3_1 = []
for i in range(5):
    models = torch.load(MODEL_PATH3_1+f'fld_{i}_quest_bert_models.pt')['model']
    model.load_state_dict(models[list(models.keys())[-1]])
    test_preds_3_1.append(eval_model(model, test_loader_2)[test_original_indices_2])
test_preds_3_1 = np.mean(test_preds_3_1, 0)
# test_preds = np.mean(test_preds_3_1, 0)

time_elapsed = time.time() - t0
print('time = {:.0f}m {:.0f}s.'.format(time_elapsed // 60, time_elapsed % 60))

Building model...


<All keys matched successfully>

<All keys matched successfully>

<All keys matched successfully>

<All keys matched successfully>

<All keys matched successfully>

time = 0m 32s.


In [25]:
# # model setup
# torch.cuda.empty_cache()

# print("Building model...")
# model = BertLastTwoClassification(bert_config, num_labels=30)
# model = model.to(device)

# # test inference
# t0 = time.time()
# torch.cuda.empty_cache()

# test_preds_3_2 = []
# for i in range(5):
#     if i==1:continue
#     models = torch.load(MODEL_PATH3_2+f'fld_{i}_quest_bert_models.pt')['model']
#     model.load_state_dict(models[list(models.keys())[-1]])
#     test_preds_3_2.append(eval_model(model, test_loader)[test_original_indices])
# # test_preds_3_2 = np.mean(test_preds_3_2, 0)
# test_preds = np.mean(test_preds_3_2, 0)

# time_elapsed = time.time() - t0
# print('time = {:.0f}m {:.0f}s.'.format(time_elapsed // 60, time_elapsed % 60))

In [26]:
# # model setup
# torch.cuda.empty_cache()

# print("Building model...")
# model = BertCLSLastTwoClassification(bert_config, num_labels=30)
# model = model.to(device)

# # test inference
# t0 = time.time()
# torch.cuda.empty_cache()

# test_preds_4 = []
# for i in range(5):
#     models = torch.load(MODEL_PATH4+f'fld_{i}_quest_bert_models.pt')['model']
#     model.load_state_dict(models[list(models.keys())[-1]])
#     test_preds_4.append(eval_model(model, test_loader)[test_original_indices])
# test_preds_4 = np.mean(test_preds_4, 0)
# # test_preds = np.mean(test_preds_4, 0)

# time_elapsed = time.time() - t0
# print('time = {:.0f}m {:.0f}s.'.format(time_elapsed // 60, time_elapsed % 60))

In [27]:
# # model setup
# torch.cuda.empty_cache()

# print("Building model...")
# model = BertLastFourClassification(bert_config, num_labels=30)
# model = model.to(device)

# # test inference
# t0 = time.time()
# torch.cuda.empty_cache()

# test_preds_5 = []
# for i in range(5):
#     models = torch.load(MODEL_PATH5+f'fld_{i}_quest_bert_models.pt')['model']
#     model.load_state_dict(models[list(models.keys())[-1]])
#     test_preds_5.append(eval_model(model, test_loader)[test_original_indices])
# # test_preds_5 = np.mean(test_preds_5, 0)
# test_preds = np.mean(test_preds_5, 0)

# time_elapsed = time.time() - t0
# print('time = {:.0f}m {:.0f}s.'.format(time_elapsed // 60, time_elapsed % 60))

In [28]:
# model setup
torch.cuda.empty_cache()

print("Building model...")
model = BertCatLastTwoClassification(bert_config, num_labels=30)
model = model.to(device)

# test inference
t0 = time.time()
torch.cuda.empty_cache()

test_preds_6 = []
for i in range(5):
    models = torch.load(MODEL_PATH6+f'fld_{i}_quest_bert_models.pt')['model']
    model.load_state_dict(models[list(models.keys())[-1]])
    test_preds_6.append(eval_model(model, test_loader)[test_original_indices])
test_preds_6 = np.mean(test_preds_6, 0)
# test_preds = np.mean(test_preds_6, 0)

time_elapsed = time.time() - t0
print('time = {:.0f}m {:.0f}s.'.format(time_elapsed // 60, time_elapsed % 60))

Building model...


<All keys matched successfully>

<All keys matched successfully>

<All keys matched successfully>

<All keys matched successfully>

<All keys matched successfully>

time = 0m 32s.


In [29]:
# model setup
torch.cuda.empty_cache()

print("Building model...")
model = BertLastTwoClassification(bert_config, num_labels=30)
model = model.to(device)

# test inference
t0 = time.time()
torch.cuda.empty_cache()

test_preds_3_3_head = []
test_preds_3_3_tail = []
test_preds_3_3_mix = []
for i in range(5):
    models = torch.load(MODEL_PATH3_3+f'fld_{i}_quest_bert_models.pt')['model']
    model.load_state_dict(models[list(models.keys())[-1]])
    
    x_test, seg_test = convert_lines(test_df, input_cols, MAX_SEQUENCE_LENGTH, tokenizer)
    test_loader, test_original_indices = prepare_loader(x_test, seg_test, batch_size=batch_size, split='test')
    test_preds_3_3_head.append(eval_model(model, test_loader)[test_original_indices])
    
    x_test, seg_test = convert_lines(test_df, input_cols, MAX_SEQUENCE_LENGTH, tokenizer, trunc_mode='tail')
    test_loader, test_original_indices = prepare_loader(x_test, seg_test, batch_size=batch_size, split='test')
    test_preds_3_3_tail.append(eval_model(model, test_loader)[test_original_indices])
    
    x_test, seg_test = convert_lines(test_df, input_cols, MAX_SEQUENCE_LENGTH, tokenizer, trunc_mode='mix')
    test_loader, test_original_indices = prepare_loader(x_test, seg_test, batch_size=batch_size, split='test')
    test_preds_3_3_mix.append(eval_model(model, test_loader)[test_original_indices])

test_preds_3_3_head = np.mean(test_preds_3_3_head, 0)
test_preds_3_3_tail = np.mean(test_preds_3_3_tail, 0)
test_preds_3_3_mix = np.mean(test_preds_3_3_mix, 0)

test_preds_3_3 = np.mean([test_preds_3_3_head, test_preds_3_3_tail, test_preds_3_3_mix], 0)
# test_preds = np.mean([test_preds_3_3_head, test_preds_3_3_tail, test_preds_3_3_mix], 0)

time_elapsed = time.time() - t0
print('time = {:.0f}m {:.0f}s.'.format(time_elapsed // 60, time_elapsed % 60))

Building model...


<All keys matched successfully>

<All keys matched successfully>

<All keys matched successfully>

<All keys matched successfully>

<All keys matched successfully>

time = 2m 53s.


In [30]:
# # model setup
# torch.cuda.empty_cache()

# print("Building model...")
# model = BertCatLastThreeClassification(bert_config, num_labels=30)
# model = model.to(device)

# # test inference
# t0 = time.time()
# torch.cuda.empty_cache()

# test_preds_7 = []
# for i in range(5):
#     models = torch.load(MODEL_PATH7+f'fld_{i}_quest_bert_models.pt')['model']
#     model.load_state_dict(models[list(models.keys())[-1]])
#     test_preds_7.append(eval_model(model, test_loader)[test_original_indices])
# # test_preds_7 = np.mean(test_preds_7, 0)
# test_preds = np.mean(test_preds_7, 0)

# time_elapsed = time.time() - t0
# print('time = {:.0f}m {:.0f}s.'.format(time_elapsed // 60, time_elapsed % 60))

In [31]:
# # model setup
# torch.cuda.empty_cache()

# print("Building model...")
# model = BertLastThreeClassification(bert_config, num_labels=30)
# model = model.to(device)

# # test inference
# t0 = time.time()
# torch.cuda.empty_cache()

# test_preds_8 = []
# for i in range(5):
#     models = torch.load(MODEL_PATH8+f'fld_{i}_quest_bert_models.pt')['model']
#     model.load_state_dict(models[list(models.keys())[-1]])
#     test_preds_8.append(eval_model(model, test_loader)[test_original_indices])
# # test_preds_8 = np.mean(test_preds_8, 0)
# test_preds = np.mean(test_preds_8, 0)

# time_elapsed = time.time() - t0
# print('time = {:.0f}m {:.0f}s.'.format(time_elapsed // 60, time_elapsed % 60))

In [32]:
# test_preds = test_preds_1*0.1 + test_preds_2*0.2 + test_preds_3*0.7 + test_preds_4*0.7 + test_preds_5*0.7
# test_preds = test_preds_2*0.21 + test_preds_3*0.3 + test_preds_4*0.4 + test_preds_5*0.09
# test_preds = test_preds_1*0.3 + test_preds_2*0.7

# test_preds = test_preds_3*0.4 + test_preds_3_1*0.2 + test_preds_3_2*0.2 + test_preds_3_3*0.2
# test_preds = test_preds_2*0.3 + test_preds_3_1*0.3 + test_preds_3*0.4

# test_preds = test_preds_3*0.5 + test_preds_6*0.5
# test_preds = test_preds_3*0.4 + test_preds_3_1*0.3 + test_preds_6*0.3
test_preds = test_preds_3_3*0.3 + test_preds_3*0.25 + test_preds_3_1*0.2 + test_preds_6*0.25

In [33]:
max_voters = [142, 30, 6, 33, 13, 4, 86, 23, 7, 142, \
              15, 9, 6, 9, 8, 7, 5, 50, 36, 149, \
              52, 24, 49, 81, 88, 64, 17, 54, 50, 127]

for i in range(test_preds.shape[1]):
    test_preds[:,i]= (test_preds[:,i] // (1/max_voters[i])) * (1/max_voters[i])

In [34]:
# # Sakami's method of post-processing: golden section search
# # https://www.kaggle.com/c/google-quest-challenge/discussion/129927

# from typing import Dict, List, Tuple
# from sklearn.base import BaseEstimator, TransformerMixin

# class OptimizedRounder(BaseEstimator, TransformerMixin):

#     def __init__(self):
#         self.threshold = [0., 1.]
#         self.ab_start = [(0., 0.2), (0.8, 1.)]

#     def fit(self, train_labels, train_preds):
#         assert train_labels.shape == train_preds.shape
#         assert train_labels.ndim == 1

#         self.best_score = self.score(train_labels, train_preds)
#         self._golden_section_search(train_labels, train_preds, 0)  # lower threshold
#         score = self.score(train_labels, train_preds)
#         if score > self.best_score + 1e-3:
#             self.best_score = score
#         else:
#             self.threshold[0] = 0.

#         self._golden_section_search(train_labels, train_preds, 1)  # higher threshold
#         score = self.score(train_labels, train_preds)
#         if score > self.best_score + 1e-3:
#             self.best_score = score
#         else:
#             self.threshold[1] = 1.

#     def _golden_section_search(self, train_labels, train_preds, idx):
#         # idx == 0 -> lower threshold search
#         # idx == 1 -> higher threshold search
#         golden1 = 0.618
#         golden2 = 1 - golden1
#         for _ in range(10):
#             a, b = self.ab_start[idx]
#             # calc losses
#             self.threshold[idx] = a
#             la = -self.score(train_labels, train_preds)
#             self.threshold[idx] = b
#             lb = -self.score(train_labels, train_preds)
#             for _ in range(20):
#                 # choose value
#                 if la > lb:
#                     a = b - (b - a) * golden1
#                     self.threshold[idx] = a
#                     la = -self.score(train_labels, train_preds)
#                 else:
#                     b = b - (b - a) * golden2
#                     self.threshold[idx] = b
#                     lb = -self.score(train_labels, train_preds)

#     def transform(self, preds):
#         transformed = np.clip(preds, *self.threshold)
#         if np.unique(transformed).size == 1:
#             return preds
#         return transformed

#     def score(self, labels, preds):
#         p = self.transform(preds)
#         score = spearmanr(labels, p)[0]
#         return score
    

# def get_scores(y_true, y_pred) -> Dict[str, float]:
#     # y_true, y_pred: np.ndarray with shape (sample_size, num_targets)
#     assert y_true.shape == y_pred.shape
#     scores = {}
#     for i, target_name in enumerate(train_df.columns[11:]):
#         scores[target_name] = spearmanr(y_true[:, i], y_pred[:, i])[0]
#     return scores

In [35]:
# thresholds = \
# defaultdict(list,
#             {'question_asker_intent_understanding': [[0.0, 1.0],
#               [0.0, 1.0],
#               [0.0, 1.0]],
#              'question_body_critical': [[0.0, 1.0], [0.0, 1.0], [0.0, 1.0]],
#              'question_conversational': [[0.15179361491144003, 1.0],
#               [0.1526515002805198, 1.0],
#               [0.1526515002805198, 1.0]],
#              'question_expect_short_answer': [[0.0, 1.0],
#               [0.0, 1.0],
#               [0.0, 1.0]],
#              'question_fact_seeking': [[0.0, 1.0], [0.0, 1.0], [0.0, 1.0]],
#              'question_has_commonly_accepted_answer': [[0.0,
#                0.8000132068576686],
#               [0.0, 0.8052740200518711],
#               [0.0, 0.8001597425885231]],
#              'question_interestingness_others': [[0.0, 1.0],
#               [0.0, 1.0],
#               [0.0, 1.0]],
#              'question_interestingness_self': [[0.0, 1.0],
#               [0.0, 1.0],
#               [0.0, 1.0]],
#              'question_multi_intent': [[0.14753784054476035, 1.0],
#               [0.13778275878897764, 1.0],
#               [0.12290187523643346, 1.0]],
#              'question_not_really_a_question': [[0.013759602164305064, 1.0],
#               [0.01802086694133006, 1.0],
#               [0.00687251255016287, 1.0]],
#              'question_opinion_seeking': [[0.0, 1.0], [0.0, 1.0], [0.0, 1.0]],
#              'question_type_choice': [[0.12734813717513502, 1.0],
#               [0.12988982368753088, 1.0],
#               [0.12362481850246866, 1.0]],
#              'question_type_compare': [[0.15529071537963646, 1.0],
#               [0.15529071537963646, 1.0],
#               [0.16294273169729784, 1.0]],
#              'question_type_consequence': [[0.08057223398703053, 1.0],
#               [0.11797798670613772, 1.0],
#               [0.12013096495104046, 1.0]],
#              'question_type_definition': [[0.168204391758552, 1.0],
#               [0.1278810398840085, 1.0],
#               [0.168204391758552, 1.0]],
#              'question_type_entity': [[0.14588181969092245, 1.0],
#               [0.12726306722893924, 1.0],
#               [0.19575783226362878, 1.0]],
#              'question_type_instructions': [[0.12299246347539239,
#                0.8107715462573686],
#               [0.07377177371375203, 0.8083782962560995],
#               [0.05334678194812586, 0.8101507557008114]],
#              'question_type_procedure': [[0.0, 1.0], [0.0, 1.0], [0.0, 1.0]],
#              'question_type_reason_explanation': [[0.09831179170683711, 1.0],
#               [0.15280740045766866, 1.0],
#               [0.0, 1.0]],
#              'question_type_spelling': [[0.009473640174145823, 1.0],
#               [0.018044454423497253, 1.0],
#               [0.018044454423497253, 1.0]],
#              'question_well_written': [[0.0, 1.0], [0.0, 1.0], [0.0, 1.0]],
#              'answer_helpful': [[0.0, 1.0], [0.0, 1.0], [0.0, 1.0]],
#              'answer_level_of_information': [[0.0, 1.0],
#               [0.0, 1.0],
#               [0.0, 1.0]],
#              'answer_plausible': [[0.0, 1.0], [0.0, 1.0], [0.0, 1.0]],
#              'answer_relevance': [[0.0, 1.0],
#               [0.0, 0.9555842698655993],
#               [0.0, 0.98346289268097]],
#              'answer_satisfaction': [[0.0, 1.0], [0.0, 1.0], [0.0, 1.0]],
#              'answer_type_instructions': [[0.069749400282935, 1.0],
#               [0.10554600900728257, 0.8946810438377845],
#               [0.055883473523577906, 0.8840116932937044]],
#              'answer_type_procedure': [[0.0, 1.0], [0.0, 1.0], [0.0, 1.0]],
#              'answer_type_reason_explanation': [[0.0, 1.0],
#               [0.0, 1.0],
#               [0.0, 1.0]],
#              'answer_well_written': [[0.0, 1.0], [0.0, 1.0], [0.0, 1.0]]})


# for i, target_name in enumerate(output_cols):
#     optimizer = OptimizedRounder()
#     optimizer.threshold = np.mean(thresholds[target_name], axis=0)
#     test_preds[:, i] = optimizer.transform(test_preds[:, i])

In [36]:
# # Post processing of column 'question_type_spelling'

# col_ind = np.where(sub_df.columns[1:] == 'question_type_spelling')[0][0]
# post_idx = np.where((test_df.category == "CULTURE") & \
#                     ((test_df.host == "english.stackexchange.com") | (test_df.host == "ell.stackexchange.com")))[0]

# pred_post = np.zeros(len(sub_df))
# pred_post[post_idx] += 0.5

# test_preds[:, col_ind] = pred_post

In [37]:
# # Post processing

# col_ind = np.where(sub_df.columns[1:] == 'answer_helpful')[0][0]
# col_rel_ind = np.where(sub_df.columns[1:] == 'answer_satisfaction')[0][0]
# col_rel_2_ind = np.where(sub_df.columns[1:] == 'answer_relevance')[0][0]
# test_preds[:, col_ind] = 0.333*(test_preds[:, col_ind] + test_preds[:, col_rel_ind] + test_preds[:, col_rel_2_ind])


# col_ind = np.where(sub_df.columns[1:] == 'answer_plausible')[0][0]
# col_rel_ind = np.where(sub_df.columns[1:] == 'answer_helpful')[0][0]
# test_preds[:, col_ind] = 0.5*(test_preds[:, col_ind] + test_preds[:, col_rel_ind])


# col_ind = np.where(sub_df.columns[1:] == 'answer_relevance')[0][0]
# col_rel_ind = np.where(sub_df.columns[1:] == 'answer_satisfaction')[0][0]
# col_rel_2_ind = np.where(sub_df.columns[1:] == 'answer_helpful')[0][0]
# test_preds[:, col_ind] = 0.333*(test_preds[:, col_ind] + test_preds[:, col_rel_ind] + test_preds[:, col_rel_2_ind])

In [38]:
# y_train = train_df[output_cols].values

# for column_ind in range(30):
#     curr_column = y_train[:, column_ind]
#     values = np.unique(curr_column)
#     map_quantiles = []
#     for val in values:
#         occurrence = np.mean(curr_column == val)
#         cummulative = sum(el['occurrence'] for el in map_quantiles)
#         map_quantiles.append({'value': val, 'occurrence': occurrence, 'cummulative': cummulative})
            
#     for quant in map_quantiles:
#         pred_col = test_preds[:, column_ind]
#         q1, q2 = np.quantile(pred_col, quant['cummulative']), np.quantile(pred_col, min(quant['cummulative'] + quant['occurrence'], 1))
#         pred_col[(pred_col >= q1) & (pred_col <= q2)] = quant['value']
#         test_preds[:, column_ind] = pred_col

In [39]:
print(str(torch.cuda.memory_allocated(device)/1e6 ) + 'M')
print(str(torch.cuda.memory_cached(device)/1e6 ) + 'M')

878.861312M
2394.947584M


In [40]:
sub_df.iloc[:, 1:] = test_preds
sub_df.to_csv('submission.csv', index=False)